In [1]:
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import random
from scipy.spatial.transform import Rotation as R
from scipy.integrate import simps


In [2]:
# Init unit axis vectors
x_h, y_h, z_h = np.array([1,0,0]), np.array([0,1,0]), np.array([0,0,1])

In [3]:
# Convenience functions init
def get_cones(pt, vec):
    """
    Return the cone with origin at pt, vector
    """
    cones = go.Cone(x=[pt[0]], y=[pt[1]], z=[pt[2]], 
                    u=[vec[0]], v=[vec[1]], w=[vec[2]], 
                    sizemode='scaled',
                    sizeref=0.1,
                    anchor='tail',
                   )
    return cones

def basis(pt, *args):
    return [get_cones(pt, v) for v in args]

In [1]:
class halfplane:
    def __init__(self, strike, dip, div_line):
        """
        strike: Strike of the half plane in degrees
        dip: Dip of the half plane in  degrees
        div_line: Vector of the line that cuts the plane in half
        """
        rad_strike = np.pi * strike / 180
        rad_dip = np.pi * dip / 180
        self.rad_strike = rad_strike
        self.rad_dip = rad_dip
        self.strike = strike
        self.dip = dip
        self.div_line = div_line
        self.normal = self.fnormal(strike, dip)
        
        self.rotation = R.from_euler('ZYX', [-strike, dip, 90], degrees=True)
        if np.dot(self.normal, div_line) > 1e-6:
            raise ValueError("div_line not in plane")
        
    def in_space(self, pt):
        """
        Returns bool value if pt is within the half plane defined by the class
        """
        # This function below evaluates z coordinate of self.div_line at point [x, y]
        z_func_line = lambda x, y: -np.dot(np.array([x, y]), self.div_line[:2]) / self.div_line[2]
        if np.dot(self.normal, pt) < 1e-6 and pt[2] < z_func_line(pt[0], pt[1]):
            return True
        else:
            return False
    
    def fnormal(self, strike, dip):
        """
        Returns the normal vector given strike and dip
        """
        deg_to_rad = lambda x: x * np.pi / 180
        r_strike = deg_to_rad(strike)
        r_dip = deg_to_rad(dip)
        n = np.array([
            np.sin(r_dip) * np.cos(r_strike),
            -np.sin(r_dip) * np.sin(r_strike),
            -np.cos(r_dip)
        ])
        return n
    
    def pai_yeoh_transform(self, electrode_pt, inverse=False):
        normal = self.fnormal(self.strike, self.dip)
        s = -np.dot(normal, electrode_pt) / np.power(normal, 2)
        closest_pt = electrode_pt + s * normal
        print(closest_point)

In [5]:
half_line = np.array([1,0,0])
myHalfplane = halfplane(270, 50, half_line)


In [ ]:
def mag_field_onplate(X, Y, Z):
    """
    X, Y, Z: Global coordinates to evaluate the BField at
    Resultant BField due to current flow in the half-plane plate from 
    an electrode that is in contact with the plate
    """
    # All coordinates that are used in this function need to be transformed into the local coordinates for the 
    # computations required
    u0 = 1.25663706e-6
    
    # We have to transform the input coordinates into the 
    x0 = self.src_x0 = 0
    z0 = self.src_z0 = 0
    
    def Bx():
        t = np.linspace(0, np.pi / 2, num=200)
        frac_bot1 = np.power(np.abs(Y) + x0 * np.sin(t), 2) + \
                    np.power(X * np.cos(t) - (Z - z0) * np.sin(t), 2)
        frac_bot2 = np.power(np.abs(Y) + x0 * np.sin(t), 2) + \
                    np.power(X * np.cos(t) + (Z - z0) * np.sin(t), 2)
        frac_top = np.abs(Y) + x0 * np.sin(t)
        y = np.cos(t) * (frac_top / frac_bot1 - frac_top / frac_bot2)
        area = simps(y, t)
        return area * np.sign(y) * -1 * u0 / 4 / np.pi ^ 2
    
    def Bz():
        t = np.linspace(0, np.pi / 2, num=200)
        frac_bot1 = np.power(np.abs(Y) + x0 * np.sin(t), 2) + \
                    np.power(X * np.cos(t) - (Z - z0) * np.sin(t), 2)
        frac_bot2 = np.power(np.abs(Y) + x0 * np.sin(t), 2) + \
                    np.power(X * np.cos(t) + (Z - z0) * np.sin(t), 2)
        frac_top = np.abs(Y) + x0 * np.sin(t)
        y = np.sin(t) * (frac_top / frac_bot1 + frac_top / frac_bot2)
        area = simps(y, t)
        return area * np.sign(Y) * u0 / 4 / np.pi ^ 2
    
    def By():
        t = np.linspace(0, np.pi / 2, num=200)
        
        frac_top1 = X * np.cos(t) - (Z - z0) * np.sin(t)
        frac_top2 = X * np.cos(t) + (Z - z0) * np.sin(t)
        frac_bot1 = np.power(np.abs(Y) + x0 * np.sin(t), 2) + \
                    np.power(X * np.cos(t) - (Z - z0) * np.sin(t), 2)
        frac_bot2 = np.power(np.abs(Y) + x0 * np.sin(t), 2) + \
                    np.power(X * np.cos(t) + (Z - z0) * np.sin(t), 2)
        
        y = frac_top1 / frac_bot1 - frac_top2 / frac_bot2
        area = simps(y, t)
        return area * u0 / 4 / np.pi ^ 2
    
    # We need to inverse transform this vector back into global coordinates 
    # **ROTATION ONLY, no translation
    unrotated_vector = np.array([Bx, By, Bz])
    
    return unrotated_vector